# Custom functions used in multiple scripts

In [2]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt


##  Function for fitting exponent, and plot scaling relation ships. 

In [2]:
def linFit(df, xvar, yvar, CI = 0.05):
    """
    Get scaling exponent using regression of the log-transformed variables. 
    Not this function drops 0 or negative values in data. 
    
    df - dataframe where the data is stored
    xvar - column name for the x variable 
    yvar - column nage for the y variable. 
    CI - define confidence interval. Default 0.05 gives 95% CI. 
    
    """
    
    df1 = df[(df[xvar] > 0) & (df[yvar] > 0)][[xvar, yvar]]
    X = np.log(df1[xvar])
    y = np.log(df1[yvar])
    
    X = sm.add_constant(X) # add constant to linear regression
    res = sm.OLS(y,X ,missing = 'drop').fit()
    #print(res.conf_int(0.05))
    out = {"b": res.params[0], "b_l" : res.conf_int(CI) [0][0], "b_u": res.conf_int(CI) [1][0], 
          "beta":res.params[1], "beta_l" : res.conf_int(CI) [0][1],"beta_u": res.conf_int(CI) [1][1],
          "rsq": res.rsquared}
          # b - intercept _l - lower bound on CI, _u - upper bound on CI, beta - slope
  
    return out, df1   

def plotScaling(df, xvar, yvar, linfit = 1, ptMarker = ".", markerSz = 5, lineMarker = "-", dispText = 1):
    """
    Plot the scaling relationship
    df - dataframe where the data is stored
    xvar - column name for the x variable 
    yvar - column nage for the y variable. 
    linfit - whether to fit the exponent and plot the scaling curve. 
    dispText - whether to display text for the scaling exponent
    """
    
    out, df1 = linFit(df, xvar, yvar)
    xdata = df1[xvar]
    ydata = df1[yvar]
    
    plt.loglog(xdata, ydata, ptMarker, alpha = 0.9, markersize = markerSz)
    
    # plot linfit 
    if linfit == 1: 
        x = np.logspace(np.log10(min(xdata)), np.log10(max(xdata)), 10)
        y = np.exp(out["b"])*x**out["beta"]
        plt.loglog(x,y , lineMarker)
        if dispText ==1:
            plt.text(min(x), max(max(y), max(ydata))*0.8 , r"$\beta = {:.2f}[{:.2f}, {:.2f}]$".format(out["beta"], out["beta_l"], out["beta_u"]))
    
    return out

In [4]:
def process_freq_table(data, ID, Function, Count):
    '''
    Process data frame where columns are:
    ID : ID of the uni
    Function: name of the function
    Count: Number of occurances of this function. 
    
    Return dataframe of ID, N (total occurances), and D (distinct number of occurances)
    '''
    # clean data 
    # incase there are multiple rows for the same function, add them. 

    data_filtered = data.groupby([ID, Function])[Count].sum().reset_index()
    
    #filter out 0 degree counts. 
    data_filtered = data_filtered[data_filtered[Count] > 0]
    
    # Count total occurances and number of functions. 
    out = data_filtered.groupby(ID).agg({Count: "sum", Function: "count"}).reset_index().rename(columns = {Count:'N',Function:"D" })
    return out
    

In [5]:
def get_rank_freq(data, Function, Count):
    """
    Get rank-frequency distribution from table where each row is a function in a unit and its count. 
    """
    

    rank_freq= data.groupby(Function)[Count].sum().reset_index().sort_values(Count, ascending = False)
    rank_freq['rank'] = np.arange(len(rank_freq))+1
    rank_freq.rename(columns = {Count: 'n'}, inplace = True)
    return rank_freq
